# Data Munging with PySpark

# Installation etc.

Setting up Spark and it's dependencies is a little tedious. 
Using Windows here. 

## Download 
* JDK (prefer 8.x, 64bit)
* Hadoop (3.2.x, at this time) - for windows, we just need Hadoop Winutils
* [Hadoop winutils (corresponding to the version of Hadoop)](https://github.com/cdarlint/winutils), [another repo](https://github.com/kontext-tech/winutils)
* [Spark (3.x, at this time)](https://spark.apache.org/downloads.html)  

## Setup environment variables 

We set these environment variables that help manage paths better.
Example variable values would look like:
* JAVA_HOME = ```C:\[Java]``` 
* HADOOP_HOME = ```C:\Hadoop\hadoop-3.2.1```
* SPARK_HOME = ```C:\Spark\spark-3.2.1-bin-hadoop3.2```  

*notice there are no backslashes in the end. This is because slashes will be added in the next step when we setup path*      

## Update system **'PATH'**

We use the variables defined above to set-up paths.  

* Java: ```%JAVA_HOME%/bin```
* Hadoop 01: ```%HADOOP_HOME%/bin```
* Hadoop 02: ```%HADOOP_HOME%/sbin``` (*sbin needed in addition to bin*)
* Spark: ```%SPARK_HOME%/bin```  
    
(*here we add backslashes before bin*)

## Patch Hadoop  

This is *needed* when Hadoop is run on Windows.

* copy the ```bin``` folder from the right version of winutils to replace ```%HADOOP_HOME%/bin```  

* copy ```hadoop-yarn-server-timelineservice-3.0.3``` from ```%HADOOP_HOME%\share\hadoop\yarn\timelineservice``` to ```%HADOOP_HOME%\share\hadoop\yarn``` (the parent directory).  

## References  

* [How to install Hadoop on Win 10](https://muhammadbilalyar.github.io/blogs/How-to-install-Hadoop-on-Window-10/)
* [Hadoop on Windows](https://github.com/MuhammadBilalYar/Hadoop-On-Window)
* [Hadoop and Spark on Windows](https://dev.to/awwsmm/installing-and-running-hadoop-and-spark-on-windows-33kc)

### (Optionally) Configure Hadoop

*only needed if you want to use hadoop as your file storage system*  

* create a folder for ```namenode```
* create a folder for ```datanode```
* four files: ```core-site.xml```, ```mapred-site.xml```, ```hdfs-site.xml```, ```yarn-site.xml``` - see code for each in the [reference repo](https://github.com/MuhammadBilalYar/Hadoop-On-Window) above.

# Setup

This boiler plate helps, esp. in Jupyter Notebook situations

In [9]:
# Step 1: initialize findspark
import findspark
findspark.init()

In [10]:
# Step 2: import pyspark
import pyspark
from pyspark.sql import SparkSession
pyspark.__version__

'3.3.0'

In [11]:
# Step 3: Create a spark session

# 'local[1]' indicates spark on 1 core on the local machine, specify the number of cores needed
# use .config("spark.some.config.option", "some-value") for additional configuration

spark = SparkSession \
    .builder \
    .master('local[1]') \
    .appName("10+ minutes to pyspark") \
    .getOrCreate()

spark

Back in the day you'd need various 'contexts' as entry points into spark functionality.  
All of this is now wrapped into a SparkSession, easy to manage.

In [12]:
# The SparkSession carries the sparkContext
spark.sparkContext

<SparkContext master=local[1] appName=10+ minutes to pyspark>

Check out the spark UI link above.  
Your local UI should launch at a link like: http://localhost:4041/jobs/

In [13]:
# before we close the notebook, stop spark, otherwise Jupyter closes, but scala-spark keep going on...
# spark.stop()

# Dataframes

## DataFrames: Create and View

In [32]:
from datetime import datetime, date
import numpy as np
import pandas as pd
from pyspark.sql import Row

In [33]:
# use a list of pyspark.sql.Row
df1 = spark.createDataFrame(
    [
        Row(a=1,b=2.,c='span a',d=date(2022,7,1),e=datetime(2022,7,1,12,0)),
        Row(a=1,b=3.,c='can a ',d=date(2022,7,2),e=datetime(2022,7,2,12,0,1)),
        Row(a=1,b=4.,c='banana',d=date(2022,7,3),e=datetime(2022,7,3,12,0,2))
    ]
)

df1

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [34]:
# df1's not been evaluated yet. It's lazy evaluation
# to eval it, we go
df1.show()

+---+---+------+----------+-------------------+
|  a|  b|     c|         d|                  e|
+---+---+------+----------+-------------------+
|  1|2.0|span a|2022-07-01|2022-07-01 12:00:00|
|  1|3.0|can a |2022-07-02|2022-07-02 12:00:01|
|  1|4.0|banana|2022-07-03|2022-07-03 12:00:02|
+---+---+------+----------+-------------------+



In [35]:
# use a list of tuples with explicit schema
df2 = spark.createDataFrame(
    [
        (2,5.,'man a',date(2022,7,1),datetime(2022,7,1,12,0)),
        (2,6.,'can a',date(2022,8,1),datetime(2022,7,2,12,0)),
        (2,7.,'manna',date(2022,9,1),datetime(2022,7,3,12,0))
    ],
    schema = 'a bigint, b double, c string, d date, e timestamp'
)
df2

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [36]:
# use spark RDDs to create a dataframe
# go to the sparksession's sparkContext to access parallelize method
rdd3 = spark.sparkContext.parallelize(
    [
        (3,5., 'main', date(2022,7,1), datetime(2022,7,1,12,0,1)),
        (3,5., 'brain', date(2022,7,1), datetime(2022,7,1,12,0,1)),
        (3,5., 'pain', date(2022,7,1), datetime(2022,7,1,12,0,1))
    ]
)

df3 = spark.createDataFrame(rdd3, schema=['a','b','c','d','e'])

df3

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [41]:
# can also use a pandas dataframe to create a spark dataframe
df4_pd = pd.DataFrame(
    {
        'a': np.random.randint(0,10, size = 3),
        'b': np.random.randn(3),
        'c': ["gandalf's manager", "said", 'no'],
        'd': [date(2022,7,1),date(2022,7,2),date(2022,7,3)],
        'e': [datetime(2022,7,1,12,0,1),datetime(2022,7,2,12,0,2),datetime(2022,7,3,12,0,3)]
    }
)

df4 = spark.createDataFrame(df4_pd)

df4.show()

+---+--------------------+-----------------+----------+-------------------+
|  a|                   b|                c|         d|                  e|
+---+--------------------+-----------------+----------+-------------------+
|  1|-0.45399653457080485|gandalf's manager|2022-07-01|2022-07-01 12:00:01|
|  0|  -2.976729687801741|             said|2022-07-02|2022-07-02 12:00:02|
|  3|  -1.086314301963997|               no|2022-07-03|2022-07-03 12:00:03|
+---+--------------------+-----------------+----------+-------------------+



In [42]:
df4_pd

,a,b,c,d,e
0,1,-0.453997,gandalf's manager,2022-07-01,2022-07-01 12:00:01
1,0,-2.976730,said,2022-07-02,2022-07-02 12:00:02
2,3,-1.086314,no,2022-07-03,2022-07-03 12:00:03


In [44]:
# use printSchema() to..., you know, it says what it does
# also don't you hate that pySpark is following the Java camelCase instead of the Python snake_case?
# yeah, what's that all about?

df1.printSchema()
df2.printSchema()
df3.printSchema()
df4.printSchema()

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [50]:
# show only x rows
df1.show(1)
# vertical - if the row is too long for horizontal display
df4.show(2, vertical=True)

+---+---+------+----------+-------------------+
|  a|  b|     c|         d|                  e|
+---+---+------+----------+-------------------+
|  1|2.0|span a|2022-07-01|2022-07-01 12:00:00|
+---+---+------+----------+-------------------+
only showing top 1 row

-RECORD 0-------------------
 a   | 1                    
 b   | -0.45399653457080485 
 c   | gandalf's manager    
 d   | 2022-07-01           
 e   | 2022-07-01 12:00:01  
-RECORD 1-------------------
 a   | 0                    
 b   | -2.976729687801741   
 c   | said                 
 d   | 2022-07-02           
 e   | 2022-07-02 12:00:02  
only showing top 2 rows



In [48]:
# collect() - collects the entire df from across all nodes to the driver
# if you don't have enough memory, here's how you crash spark
# careful is the word
df3.collect()

[Row(a=3, b=5.0, c='main', d=datetime.date(2022, 7, 1), e=datetime.datetime(2022, 7, 1, 12, 0, 1)),
 Row(a=3, b=5.0, c='brain', d=datetime.date(2022, 7, 1), e=datetime.datetime(2022, 7, 1, 12, 0, 1)),
 Row(a=3, b=5.0, c='pain', d=datetime.date(2022, 7, 1), e=datetime.datetime(2022, 7, 1, 12, 0, 1))]

In [55]:
# Pandas used to have a take() method, deprecated now
# take() in spark extracts the first n rows of a dataframe
df2.take(2)

[Row(a=2, b=5.0, c='man a', d=datetime.date(2022, 7, 1), e=datetime.datetime(2022, 7, 1, 12, 0)),
 Row(a=2, b=6.0, c='can a', d=datetime.date(2022, 8, 1), e=datetime.datetime(2022, 7, 2, 12, 0))]

In [58]:
# same as take() but returns the last n rows of a dataframe
df2.tail(2)

[Row(a=2, b=6.0, c='can a', d=datetime.date(2022, 8, 1), e=datetime.datetime(2022, 7, 2, 12, 0)),
 Row(a=2, b=7.0, c='manna', d=datetime.date(2022, 9, 1), e=datetime.datetime(2022, 7, 3, 12, 0))]

In [59]:
# hey you want another way of crashing the spark driver?
# convert the spark dataframe to a pandas dataframe
# it'll collect all data from all workers into the driver
df3.toPandas()

,a,b,c,d,e
0,3,5.0,main,2022-07-01,2022-07-01 12:00:01
1,3,5.0,brain,2022-07-01,2022-07-01 12:00:01
2,3,5.0,pain,2022-07-01,2022-07-01 12:00:01
